In [6]:
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import math
from statistics import median
import pandas as pd
import glob
import os
import nest_asyncio
import os.path; from os import path
import pickle as pkl

# import gspread
nest_asyncio.apply()
from tardis_dev import datasets
from scipy.optimize import curve_fit

In [10]:
COIN_NAME = "LINK"
LEVELS = [0, 3, 6] 
TICKS = 70000 # amount of trades
LOC = r"C:\Users\Temp_Student_001\Documents\Research\market orders w.r.t LOB"
START_DATE = '2023-02-15'
END_DATE = '2023-02-16'
TRADES = 100000

### IF YOU WANT TO RUN ONE COIN RU
# grab 30 levels of orderbook tick data
# can take a little time
# dfOB = getOB(COIN_NAME, START_DATE, END_DATE, TICKS, loc = LOC)

LOC = r"C:\Users\Temp_Student_001\Documents\Research\market orders w.r.t LOB"
# dfTrades = getTrades(COIN_NAME, START_DATE, END_DATE, TRADES, loc = LOC)

# build the aggregate OB
# this can take a little time as well
# df_aggrOB = getAggrOB(dfOB)

# plot
# plot_it(dfOB, df_aggrOB, dfTrades, TICKS, TRADES, LEVELS, COIN_NAME, save = True)

### MULTIPLE COINS
# if you want to run multiple plots in one go
# in this manner the plots will slowly roll out one by one 
COIN_NAMES = ["ADA", "MATIC", "LINK", "DOGE", "SHIB", "XLM", "MANA", "SOL", "LTC", "USDT"]

#for i in COIN_NAMES:
#    dfTrades = getTrades(i, START_DATE, END_DATE, TRADES, loc = LOC)
#    dfOB = getOB(i, START_DATE, END_DATE, TICKS, loc = LOC)
#    df_aggrOB = getAggrOB(dfOB)
#    plot_it(dfOB, df_aggrOB, dfTrades, TICKS, TRADES, LEVELS, i, save = False)


In [2]:
def getOB(name, startdate, enddate, ticks, loc, download_only = False):
    filename = os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_book_snapshot_25_" + startdate + "_" + name + "-USD.csv.gz")
    if not path.exists(filename):
        print("downloading data orderbook")
        datasets.download(
        exchange="coinbase",
        data_types=[
            "book_snapshot_25"],
        from_date = startdate,
        to_date = enddate,
        symbols = [name + "-USD"],
        download_dir = loc + "\\data\\" + name + "USD",
        api_key= "TD.CTo-GFKD9yJojKmk.0fsYIehJ-wDD1e-.FVe4814H5x7BkM7.uRoxlXUJQmcdRPz.HWyzjjLu4LiqBi4.sF6W",
        )
        
        orderbook_database =  glob.glob(os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_book_snapshot_25*" + startdate + "_" + name + "-USD.csv.gz"))
        df_orderbook = pd.concat((pd.read_csv(f) for f in orderbook_database), ignore_index=True)
        df_orderbook['timestamp'] = pd.to_datetime(df_orderbook.timestamp * 1000) #removed the nrows in the pd.read_csv as an argument
        df_orderbook.sort_values(by='timestamp', inplace=True)
    else:
        print("data Orderbook already exists!")

    if not download_only:
        orderbook_database =  glob.glob(os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_book_snapshot_25*" + startdate + "_" + name + "-USD.csv.gz"))
        df_orderbook = pd.concat((pd.read_csv(f) for f in orderbook_database), ignore_index=True)
        df_orderbook['timestamp'] = pd.to_datetime(df_orderbook.timestamp * 1000)
        df_orderbook.sort_values(by='timestamp', inplace=True)
        print(df_orderbook)
        return df_orderbook




In [4]:
def getTrades(name, startdate, enddate, trades, loc, download_only = False):
    print("getTrades")
    filename_trades = os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_trades_" + startdate + "_" + name + "-USD.csv.gz")
    filename_quotes = os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_quotes_" + startdate + "_" + name + "-USD.csv.gz")
    if not path.exists(filename_trades) or not path.exists(filename_quotes):
        print("downloading data..")
        datasets.download(
        exchange="coinbase",
        data_types=["quotes"],
        from_date = startdate,
        to_date = enddate,
        symbols = [name + "-USD"],
        download_dir = loc + "\\data\\" + name + "USD",
        api_key= "TD.CTo-GFKD9yJojKmk.0fsYIehJ-wDD1e-.FVe4814H5x7BkM7.uRoxlXUJQmcdRPz.HWyzjjLu4LiqBi4.sF6W",
        )
        print("downloading data trades", name)
        datasets.download(
        exchange="coinbase",
        data_types=[
            "trades"],
        from_date = startdate,
        to_date = enddate,
        symbols = [name + "-USD"],
        download_dir = loc + "\\data\\" + name + "USD",
        api_key= "TD.CTo-GFKD9yJojKmk.0fsYIehJ-wDD1e-.FVe4814H5x7BkM7.uRoxlXUJQmcdRPz.HWyzjjLu4LiqBi4.sF6W",
        )

        quote_database = glob.glob(os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_quotes*" + startdate + "_" + name + "-USD.csv.gz"))
        df_quotes = pd.concat((pd.read_csv(f) for f in quote_database), ignore_index=True)
        df_quotes['timestamp'] = pd.to_datetime(df_quotes.timestamp * 1000)
        df_quotes(by='timestamp', inplace=True)
        df_quotes['midprice'] = (df_quotes['ask_price'] + df_quotes['bid_price'])/2

        trade_database =  glob.glob(os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_trades*" + startdate + "_" + name + "-USD.csv.gz"))

        df_trades = pd.concat((pd.read_csv(f, nrows=trades) for f in trade_database), ignore_index=True)
        df_trades['timestamp'] = pd.to_datetime(df_trades.timestamp * 1000)
        df_trades.sort_values(by='timestamp', inplace=True)
        df_trades = df_trades[df_trades['timestamp'] >= df_quotes.loc[0, 'timestamp']]
        df_trades.index = range(len(df_trades))

        print(df_trades)
        return df_trades
    
    else:
        print("data Trades already exists")

    if not download_only:
        quote_database = glob.glob(os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_quotes*" + startdate + "_" + name + "-USD.csv.gz"))
        df_quotes = pd.concat((pd.read_csv(f) for f in quote_database), ignore_index=True)
        df_quotes['timestamp'] = pd.to_datetime(df_quotes.timestamp * 1000)
        df_quotes.sort_values(by='timestamp', inplace=True)
        df_quotes['midprice'] = (df_quotes['ask_price'] + df_quotes['bid_price'])/2

        trade_database =  glob.glob(os.path.join(loc + "\\data\\" + name + "USD\\", "coinbase_trades*" + startdate + "_" + name + "-USD.csv.gz"))
        df_trades = pd.concat((pd.read_csv(f, nrows=trades) for f in trade_database), ignore_index=True)
        df_trades['timestamp'] = pd.to_datetime(df_trades.timestamp * 1000)
        df_trades.sort_values(by='timestamp', inplace=True)
        df_trades = df_trades[df_trades['timestamp'] >= df_quotes.loc[0, 'timestamp']]
        df_trades.index = range(len(df_trades))

        print(df_trades)
        return df_trades       



In [23]:
#dfOB = getOB(COIN_NAME, START_DATE, END_DATE, TICKS, loc = LOC)
dfTrades = getTrades(COIN_NAME, START_DATE, END_DATE, TRADES, loc = LOC)


getTrades
data Trades already exists
       exchange    symbol                  timestamp   local_timestamp  \
0      coinbase  LINK-USD 2023-02-15 00:00:05.425272  1676419205470926   
1      coinbase  LINK-USD 2023-02-15 00:00:05.536406  1676419205585745   
2      coinbase  LINK-USD 2023-02-15 00:00:05.536406  1676419205585784   
3      coinbase  LINK-USD 2023-02-15 00:00:05.576493  1676419205628480   
4      coinbase  LINK-USD 2023-02-15 00:00:05.894821  1676419205938536   
...         ...       ...                        ...               ...   
33051  coinbase  LINK-USD 2023-02-15 23:59:39.091554  1676505579133681   
33052  coinbase  LINK-USD 2023-02-15 23:59:49.985446  1676505590027999   
33053  coinbase  LINK-USD 2023-02-15 23:59:49.985446  1676505590028012   
33054  coinbase  LINK-USD 2023-02-15 23:59:49.985446  1676505590027991   
33055  coinbase  LINK-USD 2023-02-15 23:59:58.687817  1676505598729933   

             id  side  price  amount  
0      62183932  sell  6.811  124.1

In [ ]:
dfTrades_adjusted = dfTrades[['timestamp','side','price','amount']]

dfTrades_adjusted['sell_amount'] = 0
dfTrades_adjusted['buy_amount'] = 0

#GEBRUIK GROUPBY IN PLAATS VAN DIE HELE CELL HIER BENEDEN!
dfTrades_adjusted.groupby(by=[timestamp])

In [58]:
#print(dfTrades)

for i in range(len(dfTrades_adjusted)):
        
        if dfTrades_adjusted['side'][i] == 'buy':
                dfTrades_adjusted['buy_amount'][i] = dfTrades_adjusted['amount'][i]
        elif dfTrades_adjusted['side'][i] == 'sell':  
                dfTrades_adjusted['sell_amount'][i] = dfTrades_adjusted['amount'][i]


        if dfTrades_adjusted['timestamp'][i] == dfTrades_adjusted['timestamp'][i+1] and i != len(dfTrades_adjusted) - 1: #last part of if statement is to avoid error for checking last trade
                j=i
                while dfTrades_adjusted['timestamp'][j] == dfTrades_adjusted['timestamp'][i+1]:
                        dfTrades_adjusted['buy_amount'][j] += dfTrades_adjusted['buy_amount'][i+1]
                        dfTrades_adjusted['buy_amount'][i+1] = 0
                        dfTrades_adjusted['sell_amount'][j] += dfTrades_adjusted['sell_amount'][i+1]
                        dfTrades_adjusted['sell_amount'][i+1] = 0
                        i+=1


print(dfTrades_adjusted)


C:\Users\Temp_Student_001\AppData\Local\Temp\ipykernel_13824\1597675821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrades_adjusted['sell_amount'] = [0] * len(dfTrades_adjusted)
C:\Users\Temp_Student_001\AppData\Local\Temp\ipykernel_13824\1597675821.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrades_adjusted['buy_amount'] = [0] * len(dfTrades_adjusted)
C:\Users\Temp_Student_001\AppData\Local\Temp\ipykernel_13824\1597675821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a 

KeyError: 33056

In [60]:
dfTrades_adjusted

,timestamp,side,price,amount,sell_amount,buy_amount
0,2023-02-15 00:00:05.425272,sell,6.811,124.17,124.17,0.00
1,2023-02-15 00:00:05.536406,sell,6.810,117.86,117.86,0.00
2,2023-02-15 00:00:05.536406,sell,6.810,159.56,159.56,0.00
3,2023-02-15 00:00:05.576493,sell,6.809,54.46,54.46,0.00
4,2023-02-15 00:00:05.894821,sell,6.808,73.56,73.56,0.00
...,...,...,...,...,...,...
33051,2023-02-15 23:59:39.091554,sell,7.283,30.00,30.00,0.00
33052,2023-02-15 23:59:49.985446,sell,7.282,39.26,39.26,0.00
33053,2023-02-15 23:59:49.985446,sell,7.282,40.03,40.03,0.00
33054,2023-02-15 23:59:49.985446,sell,7.282,45.20,45.20,0.00


In [12]:
df_trades = df_trades[3800:5000]
df_trades.index = range(len(df_trades))

In [ ]:
    dfOBagg['sell_amount'] = 0
    dfOBagg['buy_amount'] = 0
    

    for i in range(len(dfTrades)):
        # print(i)
        # bestTimestamp = current_timestamp

        market_order = dfTrades['timestamp'][i]
        # current_tick = dfOB[(dfOB['timestamp'] < market_order)].iloc[-1]
        # dfOB[(dfOB['timestamp'] < market_order)].iloc[-1]
        current_index = dfOB.index[(dfOB['timestamp'] < market_order)][-1]
        list.append(current_index)
        # print(last_index)
        # bestTimestamp = last_tick['timestamp']
        if dfTrades['side'][i] == 'buy':
            # print(dfTrades['amount'][i])
            # current_tick['buy_amount'] = dfTrades['amount'][i]
           dfOBagg.iloc[current_index, dfOBagg.columns.get_loc('buy_amount')] += dfTrades['amount'][i]
        elif dfTrades['side'][i] == 'sell':
           dfOBagg.iloc[current_index, dfOBagg.columns.get_loc('sell_amount')] -= dfTrades['amount'][i]